In [160]:
import pandas
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack
# обучающая выборка
dataSet_Tr = pandas.read_csv('salary-train.csv')
X_train = dataSet_Tr.iloc[:,0:3]   # признаки - первые три столбца
#print(X_train)
Y_train = dataSet_Tr.iloc[:,3]   # целевой вектор
#print(Y_train) 
dataSet_Test = pandas.read_csv('salary-test-mini.csv')
X_test = dataSet_Test.iloc[:,0:]   # признаки - первые три столбца
print(X_test)
#Y_test = dataSet_Test.iloc[:,3]   # целевой вектор
#print(Y_test.shape) 

                                     FullDescription LocationNormalized  \
0  We currently have a vacancy for an HR Project ...      Milton Keynes   
1  A Web developer opportunity has arisen with an...         Manchester   

  ContractTime  SalaryNormalized  
0     contract               NaN  
1    permanent               NaN  


In [161]:
X_train['FullDescription'] = X_train['FullDescription'].str.lower()
X_train['FullDescription'] = X_train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
X_test['FullDescription'] = X_test['FullDescription'].str.lower()
X_test['FullDescription'] = X_test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

In [162]:
vectorizer = TfidfVectorizer(min_df = 5)    # вычисление признаков TF-IDF
X_train_vec = vectorizer.fit_transform(X_train['FullDescription'])
X_test_vec = vectorizer.transform(X_test['FullDescription'])
#print(vectorizer.get_feature_names())
X_train['LocationNormalized'].fillna('nan', inplace=True)
X_test['LocationNormalized'].fillna('nan', inplace=True)
X_train['ContractTime'].fillna('nan', inplace=True)
X_test['ContractTime'].fillna('nan', inplace=True)

In [163]:
enc = DictVectorizer()
X_train_categ = enc.fit_transform(X_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(X_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [164]:
Xh_Tr = hstack((X_train_vec, X_train_categ))
Xh_Test = hstack((X_test_vec, X_test_categ))

In [165]:
clf = Ridge(alpha=1, random_state=241)
clf.fit(Xh_Tr, Y_train)
prediction = clf.predict(Xh_Test)
prediction

array([56567.05628269, 37140.50652386])